In [1]:
import os

In [2]:
%pwd

'e:\\Amazon-Data-Science-Book\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Amazon-Data-Science-Book'

In [5]:
import joblib

In [6]:
model = joblib.load("artifacts/training/model.h5")


In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data: Path
    metric_file_path: Path


In [8]:
from pathlib import Path
from src.cnnClassifier.entity.config_entity import EvaluationConfig

def get_evaluation_config(self) -> EvaluationConfig:
    return EvaluationConfig(
        model_path=Path("artifacts/training/model.pkl"),
        training_data=Path(
            self.config.data_ingestion.unzip_dir
        ) / "amazon_data_science_books_cleaned.csv",
        metric_file_path=Path("artifacts/evaluation/metrics.txt")
    )


In [9]:
import pandas as pd
import joblib
from pathlib import Path
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)


class Evaluation:
    def __init__(self, config):
        self.config = config
        self.eval_dir = Path("artifacts/evaluation")
        self.eval_dir.mkdir(parents=True, exist_ok=True)

    def evaluate(self):
        # Load trained model
        model = joblib.load(self.config.model_path)

        # Load dataset
        df = pd.read_csv(self.config.training_data)

        X = df.iloc[:, 0].astype(str)
        y = df.iloc[:, -1]

        y_pred = model.predict(X)

        # -------- Metrics --------
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(
            y, y_pred, average="weighted", zero_division=0
        )
        recall = recall_score(
            y, y_pred, average="weighted", zero_division=0
        )
        f1 = f1_score(
            y, y_pred, average="weighted", zero_division=0
        )

        # -------- Print --------
        print("\n📊 Evaluation Metrics")
        print("Accuracy :", accuracy)
        print("Precision:", precision)
        print("Recall   :", recall)
        print("F1-score :", f1)

        report = classification_report(
            y, y_pred, zero_division=0
        )
        cm = confusion_matrix(y, y_pred)

        print("\n📄 Classification Report")
        print(report)

        print("\n🧩 Confusion Matrix")
        print(cm)

        # -------- Save to artifacts --------
        self._save_metrics(accuracy, precision, recall, f1)
        self._save_report(report)
        self._save_confusion_matrix(cm)

    def _save_metrics(self, accuracy, precision, recall, f1):
        with open(self.eval_dir / "metrics.txt", "w") as f:
            f.write(f"Accuracy  : {accuracy}\n")
            f.write(f"Precision : {precision}\n")
            f.write(f"Recall    : {recall}\n")
            f.write(f"F1-score  : {f1}\n")

    def _save_report(self, report: str):
        with open(self.eval_dir / "classification_report.txt", "w", encoding="utf-8") as f:
            f.write(report)

    def _save_confusion_matrix(self, cm):
        with open(self.eval_dir / "confusion_matrix.txt", "w") as f:
            f.write(str(cm))


In [10]:
from src.cnnClassifier.config.configuration import ConfigurationManager
from src.cnnClassifier.components.evaluation import Evaluation

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()

    evaluator = Evaluation(config=eval_config)
    evaluator.evaluate()

except Exception as e:
    raise e


[2025-12-19 12:42:32,906: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-19 12:42:32,906: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-19 12:42:32,906: INFO: common: created directory at: artifacts]

📊 Evaluation Metrics
Accuracy : 0.7994269340974212
Precision: 0.7163562559694364
Recall   : 0.7994269340974212
F1-score : 0.741642788920726

📄 Classification Report
                                                                                                                                                                                                                                                                                                                                                                                                                                              precision    recall  f1-score   support

                                                                                                                   

c:\Users\hp\anaconda3\envs\endM\lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
c:\Users\hp\anaconda3\envs\endM\lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
c:\Users\hp\anaconda3\envs\endM\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\hp\anaconda3\envs\endM\lib\site-packages\sklearn\metrics\_classification.py:99: UserWar